In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Parkinsons_Disease"
cohort = "GSE72267"

# Input paths
in_trait_dir = "../../input/GEO/Parkinsons_Disease"
in_cohort_dir = "../../input/GEO/Parkinsons_Disease/GSE72267"

# Output paths
out_data_file = "../../output/preprocess/Parkinsons_Disease/GSE72267.csv"
out_gene_data_file = "../../output/preprocess/Parkinsons_Disease/gene_data/GSE72267.csv"
out_clinical_data_file = "../../output/preprocess/Parkinsons_Disease/clinical_data/GSE72267.csv"
json_path = "../../output/preprocess/Parkinsons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Blood transcriptomics of drug-naïve sporadic Parkinson’s disease patients"
!Series_summary	"Parkinson's disease (PD) is a chronic progressive neurodegenerative disorder that is clinically defined in terms of motor symptoms. These are preceded by prodromal non-motor manifestations that prove the systemic nature of the disease. Identifying genes and pathways altered in living patients provide new information on the diagnosis and pathogenesis of sporadic PD. We study changes in gene expression in the blood of 40 sporadic PD patients and 20 healthy controls (“Discovery set”) by taking advantage of the Affymetrix platform. Patients were at the onset of motor symptoms and before initiating any pharmacological treatment. By applying Ranking-Principal Component Analysis, PUMA and Significance Analysis of Microarrays, gene expression profiling discriminates patients from healthy controls and identifies differentially expressed genes in blood. The majority 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at the background information, this dataset is about blood transcriptomics (gene expression)
# of Parkinson's disease patients on the Affymetrix platform, which is a microarray platform for gene expression.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Parkinson's Disease): From the sample characteristics dictionary, key 0 contains diagnosis information
trait_row = 0

# For age: Not available in the sample characteristics dictionary
age_row = None

# For gender: Not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
# Convert trait (Parkinson's Disease) - Binary (0: Healthy, 1: Parkinson's disease)
def convert_trait(value):
    if isinstance(value, str):
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if value.lower() == "healthy":
            return 0
        elif value.lower() == "parkinson's disease":
            return 1
    return None

# Since age and gender are not available, we define placeholder functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the library function
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data, 
        trait=trait, 
        trait_row=trait_row, 
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features_df)
    print("Preview of clinical features dataframe:")
    print(preview)
    
    # Save the clinical features dataframe
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features dataframe:
{'GSM1859079': [0.0], 'GSM1859080': [1.0], 'GSM1859081': [1.0], 'GSM1859082': [0.0], 'GSM1859083': [0.0], 'GSM1859084': [1.0], 'GSM1859085': [0.0], 'GSM1859086': [1.0], 'GSM1859087': [0.0], 'GSM1859088': [1.0], 'GSM1859089': [1.0], 'GSM1859090': [0.0], 'GSM1859091': [0.0], 'GSM1859092': [1.0], 'GSM1859093': [0.0], 'GSM1859094': [0.0], 'GSM1859095': [1.0], 'GSM1859096': [1.0], 'GSM1859097': [0.0], 'GSM1859098': [1.0], 'GSM1859099': [0.0], 'GSM1859100': [1.0], 'GSM1859101': [0.0], 'GSM1859102': [1.0], 'GSM1859103': [1.0], 'GSM1859104': [0.0], 'GSM1859105': [1.0], 'GSM1859106': [0.0], 'GSM1859107': [0.0], 'GSM1859108': [1.0], 'GSM1859109': [0.0], 'GSM1859110': [1.0], 'GSM1859111': [0.0], 'GSM1859112': [1.0], 'GSM1859113': [0.0], 'GSM1859114': [1.0], 'GSM1859115': [1.0], 'GSM1859116': [0.0], 'GSM1859117': [1.0], 'GSM1859118': [1.0], 'GSM1859119': [1.0], 'GSM1859120': [1.0], 'GSM1859121': [1.0], 'GSM1859122': [1.0], 'GSM1859123': [1.0], 'GSM1859124': 

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (like '1007_s_at', '1053_at')
# rather than standard human gene symbols (which would look like 'BRCA1', 'TP53', etc.)
# Affymetrix probe IDs need to be mapped to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene annotation data, I can see that:
# - The 'ID' column contains Affymetrix probe IDs (e.g., '1007_s_at') that match our gene expression data index
# - The 'Gene Symbol' column contains the gene symbols we need (e.g., 'DDR1 /// MIR4640')

# 2. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print a preview of the mapping to confirm
print("Gene mapping preview (first 5 rows):")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print preview of gene-level expression data
print("\nGene expression data after mapping (first 5 genes):")
print(preview_df(gene_data))

# Optional: Count how many genes we have after mapping
print(f"\nNumber of genes after mapping: {len(gene_data)}")


Gene mapping preview (first 5 rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}

Gene expression data after mapping (first 5 genes):
{'GSM1859079': [3.740457156, 6.18821045, 6.12347017, 2.933598399, 8.492357612], 'GSM1859080': [3.589667857, 6.343765541, 6.377846963, 2.677448329, 8.736387558], 'GSM1859081': [4.001616363, 5.564381559, 6.764953134, 3.334060591, 8.184671939], 'GSM1859082': [3.922479565, 5.831597337, 6.561627019, 2.927740117, 8.245925089], 'GSM1859083': [3.825040943, 6.409659658, 6.392594146, 2.773846183, 8.183062402], 'GSM1859084': [3.727894171, 5.616698925, 6.84178821, 2.694206178, 9.266868314], 'GSM1859085': [3.174143042, 5.713348166, 6.306452396, 2.758840312, 8.387619897], 'GSM1859086': [3.900209372, 5.742100387, 6.616950751, 3.32203522, 8.2486853], 'GSM1859087': [3.412826064, 5.946201536, 6.509076895, 2.864345406, 8.035643341], 'GSM1859088': [3.687393447, 5.838446028, 6.435257523, 2

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using the NCBI Gene database
# We already have gene_data with proper mapping from Step 6, continue from there
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Number of genes after normalization: {len(normalized_gene_data)}")

# 2. Load clinical data and link it with gene expression data
# Load the clinical data file that was saved in a previous step
clinical_data_file = out_clinical_data_file
clinical_df = pd.read_csv(clinical_data_file)

# Check the column structure of clinical data
print(f"Clinical data columns: {clinical_df.columns}")

# Convert clinical_df to the right format (samples as columns)
if 'Unnamed: 0' in clinical_df.columns:
    clinical_df = clinical_df.set_index('Unnamed: 0')
else:
    # If the index column hasn't been automatically named, use the first column
    clinical_df = clinical_df.set_index(clinical_df.columns[0])

# Rename the index to make sure it's labeled as the trait
clinical_df.index.name = trait
clinical_df = clinical_df.T

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(f"Linked data columns (first 10): {list(linked_data.columns[:10])}")

# 3. Handle missing values systematically
# Find the actual trait column name in the linked data
if trait in linked_data.columns:
    trait_column = trait
else:
    # Check the first few column names to locate the trait
    print(f"First 5 columns of linked data: {list(linked_data.columns[:5])}")
    # Use the first column as the trait column - this is typically correct for GEO datasets
    trait_column = linked_data.columns[0]
    print(f"Using '{trait_column}' as the trait column")

linked_data = handle_missing_values(linked_data, trait_column)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Evaluate whether the trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait_column)

# 5. Validate the dataset quality and save metadata
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Parkinson's Disease cohort with blood transcriptomics data."
)

# 6. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("The dataset was deemed not usable. Data not saved to output file.")

Normalized gene expression data saved to ../../output/preprocess/Parkinsons_Disease/gene_data/GSE72267.csv
Number of genes after normalization: 13542
Clinical data columns: Index(['GSM1859079', 'GSM1859080', 'GSM1859081', 'GSM1859082', 'GSM1859083',
       'GSM1859084', 'GSM1859085', 'GSM1859086', 'GSM1859087', 'GSM1859088',
       'GSM1859089', 'GSM1859090', 'GSM1859091', 'GSM1859092', 'GSM1859093',
       'GSM1859094', 'GSM1859095', 'GSM1859096', 'GSM1859097', 'GSM1859098',
       'GSM1859099', 'GSM1859100', 'GSM1859101', 'GSM1859102', 'GSM1859103',
       'GSM1859104', 'GSM1859105', 'GSM1859106', 'GSM1859107', 'GSM1859108',
       'GSM1859109', 'GSM1859110', 'GSM1859111', 'GSM1859112', 'GSM1859113',
       'GSM1859114', 'GSM1859115', 'GSM1859116', 'GSM1859117', 'GSM1859118',
       'GSM1859119', 'GSM1859120', 'GSM1859121', 'GSM1859122', 'GSM1859123',
       'GSM1859124', 'GSM1859125', 'GSM1859126', 'GSM1859127', 'GSM1859128',
       'GSM1859129', 'GSM1859130', 'GSM1859131', 'GSM1859